In [6]:
import bs4
import urllib.parse
import csv
import lxml


In [7]:
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
from datetime import datetime
from lxml import html, etree

## Fetching the 10Q link

In [8]:
base_url = 'https://www.sec.gov'
cik = "51143"
acc_num = "000005114313000007"
acc_num_index = acc_num[0:10]+"-"+acc_num[10:12]+"-"+acc_num[12:]+"-index.html"
url_rendered = base_url + "/Archives/edgar/data/" + cik + "/" + acc_num +"/" + acc_num_index
form_url = base_url + "/" + cik + "/" + acc_num
print(url_rendered)

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.html


In [9]:
uCLient = ureq(url_rendered)
page_html=uCLient.read()

In [10]:
page_soup = soup(page_html, 'html.parser')
divs = page_soup.find('table',summary="Document Format Files")
url2=divs.find_all('tr')[1].find_all('td')[2].find('a')['href']

## Fetching tables

In [11]:
my_url2=urllib.parse.urljoin(base_url, url2)
my_url2

'https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm'

In [12]:
uCLient2 = ureq(my_url2)
page_html2=uCLient2.read()
page_soup2 = soup(page_html2, 'html.parser')

In [56]:
for link in page_soup2.find_all('a'):
    print(link.get_text())
    print(link.get('href'))
    print('end')


None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end

None
end
Part I - Financial Information:
#Part1FinInfo
end

None
end

None
end
Item
  1. Consolidated Financial Statements:
#ConsolidatedFS
end

None
end
Consolidated Statement of Earnings for the three and nine months ended
  September 30, 2013 and 2012
#ConsolEarnings
end

None
end
Consolidated Statement of Comprehensive Income for the three and nine
  months ended September 30, 2013 and 2012
#ConsolComprehensiveIncome
end

None
end

None
end
Consolidated
  Statement of Financial Position at September 30, 2013 and December 31, 2012
#ConsolBS
end

None
end

None
end
Consolidated
  Statement of Cash Flows for the nine months ended September 30, 2013 and 2012
#ConsolCF
end

None
end

None
end
Consolidated
  Statement of Changes in Equity for the nine months en

In [58]:
link_list=[]
link_text=[]
for link in page_soup2.find_all('a'):
    href_link=link.get('href')
    href_text=link.get_text()
   
    if href_link is not None:
        href_link=str(href_link).strip('#')
        link_list.append(href_link)
        link_text.append(href_text)
print(link_list)
print(link_text)


['Part1FinInfo', 'ConsolidatedFS', 'ConsolEarnings', 'ConsolComprehensiveIncome', 'ConsolBS', 'ConsolCF', 'ConsolEquity', 'NotesToConsolFS', 'MDA', 'Controls', 'OtherInfo', 'LegalProceedings', 'Unregistered', 'Exhibits']
['Part I - Financial Information:', 'Item\n  1. Consolidated Financial Statements:', 'Consolidated Statement of Earnings for the three and nine months ended\n  September 30, 2013 and 2012', 'Consolidated Statement of Comprehensive Income for the three and nine\n  months ended September 30, 2013 and 2012', 'Consolidated\n  Statement of Financial Position at September 30, 2013 and December 31, 2012', 'Consolidated\n  Statement of Cash Flows for the nine months ended September 30, 2013 and 2012', 'Consolidated\n  Statement of Changes in Equity for the nine months ended September 30, 2013\n  and 2012', 'Notes\n  to Consolidated Financial Statements', 'Item\n  2. Management’s Discussion and Analysis of Results of Operations and\n  Financial Condition', 'Item 4. Controls and

In [15]:
pages=page_soup2.find("a",{'name':'Part1FinInfo'})
page1=pages.parent.parent.parent
print(page1)
table_element = page1.select('div[align="left"]')
table_element

<div bclpageborder="" style="PADDING-TOP: 0in; PADDING-BOTTOM: 0in; WIDTH: 100%; PADDING-RIGHT: 0in; PADDING-LEFT: 0in">
<div bclheader="">
<p align="center" style="border:none;margin:0in;margin-bottom:.0001pt;mso-border-shadow:yes;mso-layout-grid-align:none;mso-padding-alt:31.0pt 31.0pt 31.0pt 31.0pt;mso-pagination:none;text-align:center;"><a name="Part1FinInfo"></a><b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.5pt;"><b><font face="Times New Roman,serif" lang="EN-US" style="font-size:10.0pt;">PART I - Financial Information</font></b></font></font></b><a name="Part1FinInfo_bclEnd"></a><b><font style="mso-bidi-font-weight:normal;"><font style="border:none;"> </font></font></b></p>
<p style="border:none;margin:0in;margin-bottom:.0001pt;mso-border-shadow:yes;mso-layout-grid-align:none;mso-padding-alt:31.0pt 31.0pt 31.0pt 31.0pt;mso-pagination:none;"><font style="border:none;"><font face="Times New Roman,serif" lang="EN-US" style="font-size:10

[<div align="left"><table border="1" cellpadding="0" cellspacing="0" style="border:none;border-collapse:collapse;width:100%;">
 <tr style="height:12.75pt;page-break-inside:avoid;">
 <td nowrap="" style="border:none;height:12.75pt;padding:0in 0in 0in 0in;" valign="bottom" width="2%">
 <p style="margin:0in;margin-bottom:.0001pt;page-break-after:avoid;page-break-before:always;"><font color="black" face="Times New Roman,serif" style="font-size:10.0pt;"> </font></p>
 </td>
 <td nowrap="" style="border:none;height:12.75pt;padding:0in 0in 0in 0in;" valign="bottom" width="34%">
 <p align="center" style="margin:0in;margin-bottom:.0001pt;page-break-after:avoid;text-align:center;"><font color="black" face="Times New Roman,serif" style="font-size:10.0pt;"> </font></p>
 </td>
 <td nowrap="" style="border:none;height:12.75pt;padding:0in 0in 0in 0in;" valign="bottom" width="1%">
 <p style="margin:0in;margin-bottom:.0001pt;page-break-after:avoid;"><font color="black" face="Times New Roman,serif" style

In [16]:
len(page1)

9

In [17]:
table_element = page1.select('div[align="left"]')
table_all_rows = []
for table in table_element:
    table_row = table.select('tr')
    for j in table_row:
        table_column = j.select('font')
        row = []
        for k in table_column:
            k_text = k.text.replace(u'\xa0',u'')
            #k_text = k_text.encode("utf-8")
            k_text = str(k_text).strip('b').strip()
            k_text= k_text.strip('"')
            print(k_text)
            row.append(k_text)
        table_all_rows.append(row)
    try:
        with open(('test1.csv'),"a") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            try:
                csvWriter.writerows(table_all_rows) 
            except:
                print("no")
    except:
        print("file open error")


















ThreeMonths Ended
  September 30,


Nine Months Ended
  September 30,
(Dollarsinmillionsexcept
  pershareamounts)


2013


2012


2013


2012
Revenue:












Services

$
14,225

$
14,626

$
42,811

$
44,279
Sales


8,987


9,642


27,735


29,424
Financing


509


479


1,506


1,500
Total
  revenue


23,720


24,747


72,052


75,203
Cost:












Services


9,098


9,515


27,950


29,285
Sales


2,975


3,242


9,108


10,003
Financing


268


258


805


784
Total
  cost


12,341


13,016


37,863


40,072
Gross
  profit


11,380


11,732


34,189


35,131
Expense
  and other income:












Selling,
  general and administrative


5,255


5,908


17,512


17,632
Research,
  development and engineering


1,468


1,534


4,661


4,722
Intellectual
  property and custom













development
  income


(191)


(303)


(621)


(847)
Other
  (income) and expense


(62)


(606)


(214)


(796)
Interest
  expense


97


124


289


350
Total
  expense and 

In [18]:
soup_part=page_soup2

In [24]:
contents=[]
for item in link_list:
    pages=page_soup2.find("a",{'name':item})
    print(pages)
    for page in pages.find_next_siblings():
        print(page)
        

<a name="Part1FinInfo"></a>
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.5pt;"><b><font face="Times New Roman,serif" lang="EN-US" style="font-size:10.0pt;">PART I - Financial Information</font></b></font></font></b>
<a name="Part1FinInfo_bclEnd"></a>
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;"> </font></font></b>
<a name="ConsolidatedFS"></a>
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.0pt;"><b><font face="Times New Roman,serif" lang="EN-US" style="font-size:10.0pt;">Item</font></b></font></font></b>
<font style="mso-bookmark:ConsolidatedFS;"><b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.5pt;"><b><font face="Times New Roman,serif" lang="EN-US" style="font-size:10.0pt;"> 1. Consolidated Financial

Statements<a name="Item1_ConsolidatedFinancialStatem_125609"></a>:</font></b></font></font></b></font>
<a name="Consolida

AttributeError: 'NoneType' object has no attribute 'find_next_siblings'

In [98]:
contents=[]
for item in link_list:
    pages=page_soup2.find("a",{'name':item})
    print(pages)
    i=iter(link_list)
    print("link list element",next(i))
    l1=next(i)
    print("link list next element",l1)
    for page in pages.find_next_siblings():
        print(page)
        #print(page("a",{'name':page}),"heeeeeeeeeyyyyyyyyyyyyy")
        #print(page.name,"hellllllllllllllll")
        
        print(page.attrs,"attributess of page")
        print(type(page.attrs))
        l2=str(page.attrs)
        print(type(l2))
        print('both elements',l1,l2)
        if l1 in l2:
            print("It is here in the loop", l1,l2)
            break

<a name="Part1FinInfo"></a>
link list element Part1FinInfo
link list next element ConsolidatedFS
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.5pt;"><b><font face="Times New Roman,serif" lang="EN-US" style="font-size:10.0pt;">PART I - Financial Information</font></b></font></font></b>
{} attributess of page
<class 'dict'>
<class 'str'>
both elements ConsolidatedFS {}
<a name="Part1FinInfo_bclEnd"></a>
{'name': 'Part1FinInfo_bclEnd'} attributess of page
<class 'dict'>
<class 'str'>
both elements ConsolidatedFS {'name': 'Part1FinInfo_bclEnd'}
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;"> </font></font></b>
{} attributess of page
<class 'dict'>
<class 'str'>
both elements ConsolidatedFS {}
<a name="ConsolidatedFS"></a>
link list element Part1FinInfo
link list next element ConsolidatedFS
<b><font style="mso-bidi-font-weight:normal;"><font style="border:none;mso-bidi-font-size:11.0pt;"><b><font face="Times New Roman,

AttributeError: 'NoneType' object has no attribute 'find_next_siblings'

In [ ]:
 #item2=list(next(item))
       # print(type(item2))
        #if str(next(item)) in str(page.attrs):
            #print("It is here", next(item),page.attrs)
        